In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/1000_records.csv')
df.head()

,Emp ID,Name Prefix,First Name,Middle Initial,Last Name,Gender,E Mail,Father's Name,Mother's Name,Mother's Maiden Name,...,SSN,Phone No.,Place Name,County,City,State,Zip,Region,User Name,Password
0,850297,Ms.,Shawna,W,Buck,F,shawna.buck@gmail.com,Rosario Buck,Keisha Buck,Hendricks,...,222-11-7603,702-771-7149,Las Vegas,Clark,Las Vegas,NV,89128,West,swbuck,ja8?k3BTF^]o@<&
1,304721,Mr.,Nathaniel,Z,Burke,M,nathaniel.burke@walmart.com,Derrick Burke,Phoebe Burke,Pugh,...,550-99-9897,231-765-6923,Irons,Lake,Irons,MI,49644,Midwest,nzburke,tPA@4Wh9Ja$+Ln
2,412317,Drs.,Elisabeth,W,Foster,F,elisabeth.foster@gmail.com,Irwin Foster,Janie Foster,Delaney,...,215-91-0004,270-749-4774,Lexington,Fayette,Lexington,KY,40501,South,ewfoster,5D.B4IGNkWJfG<~
3,621375,Mrs.,Briana,C,Lancaster,F,briana.lancaster@yahoo.com,Jeffrey Lancaster,Shelby Lancaster,Weiss,...,534-71-1053,219-623-8216,Munster,Lake,Munster,IN,46321,Midwest,bclancaster,I%?JDM;pW
4,787549,Hon.,Estella,L,Potter,F,estella.potter@gmail.com,Booker Potter,Katelyn Potter,Pate,...,644-29-6396,907-677-8486,Fairbanks,Fairbanks North Star,Fairbanks,AK,99709,West,elpotter,j4jkEXfC&Rh


Observations:

1. There are so many columns which will not be very useful for the model. Those will be removed.
2. There are categorical as well as numeric variables


In [3]:
df.columns

Index(['Emp ID', 'Name Prefix', 'First Name', 'Middle Initial', 'Last Name',
       'Gender', 'E Mail', 'Father's Name', 'Mother's Name',
       'Mother's Maiden Name', 'Date of Birth', 'Time of Birth', 'Age in Yrs.',
       'Weight in Kgs.', 'Date of Joining', 'Quarter of Joining',
       'Half of Joining', 'Year of Joining', 'Month of Joining',
       'Month Name of Joining', 'Short Month', 'Day of Joining',
       'DOW of Joining', 'Short DOW', 'Age in Company (Years)', 'Salary',
       'Last % Hike', 'SSN', 'Phone No. ', 'Place Name', 'County', 'City',
       'State', 'Zip', 'Region', 'User Name', 'Password'],
      dtype='object')

Observations:
  The following columns may not contribute to the model. So will be dropped.
  
 ['Emp ID', 'Name Prefix', 'First Name', 'Middle Initial', 'Last Name','E Mail', 'Father's Name', 'Mother's Name',
       'Mother's Maiden Name', 'Date of Birth', 'Time of Birth','Weight in Kgs.', 'Date of Joining','Month of Joining',
       'Month Name of Joining', 'Short Month', 'Day of Joining',
       'DOW of Joining', 'Short DOW', 'SSN', 'Phone No. ', 'User Name', 'Password']
       
Further investigation will be done on the remaining columns in the dataset.

In [11]:
data = df.drop(['Emp ID', 'Name Prefix', 'First Name', 'Middle Initial', 'Last Name','E Mail', "Father's Name", "Mother's Name", "Mother's Maiden Name", 'Date of Birth', 'Time of Birth','Weight in Kgs.', 'Date of Joining','Month of Joining', 'Month Name of Joining', 'Short Month', 'Day of Joining', 'DOW of Joining', 'Short DOW', 'SSN', 'Phone No. ', 'User Name', 'Password'], axis=1)
data.head()

,Gender,Age in Yrs.,Quarter of Joining,Half of Joining,Year of Joining,Age in Company (Years),Salary,Last % Hike,Place Name,County,City,State,Zip,Region
0,F,45.66,Q4,H2,2010,6.61,119090,17%,Las Vegas,Clark,Las Vegas,NV,89128,West
1,M,23.76,Q2,H1,2017,0.26,117991,14%,Irons,Lake,Irons,MI,49644,Midwest
2,F,22.68,Q1,H1,2017,0.34,161045,28%,Lexington,Fayette,Lexington,KY,40501,South
3,F,41.70,Q1,H1,2003,14.43,142616,16%,Munster,Lake,Munster,IN,46321,Midwest
4,F,22.39,Q4,H2,2016,0.79,135706,11%,Fairbanks,Fairbanks North Star,Fairbanks,AK,99709,West


In [12]:
target = data['Salary']
data.drop(labels=['Salary'], axis=1, inplace = True)
data.insert(13, 'Salary', target)
data.head()

,Gender,Age in Yrs.,Quarter of Joining,Half of Joining,Year of Joining,Age in Company (Years),Last % Hike,Place Name,County,City,State,Zip,Region,Salary
0,F,45.66,Q4,H2,2010,6.61,17%,Las Vegas,Clark,Las Vegas,NV,89128,West,119090
1,M,23.76,Q2,H1,2017,0.26,14%,Irons,Lake,Irons,MI,49644,Midwest,117991
2,F,22.68,Q1,H1,2017,0.34,28%,Lexington,Fayette,Lexington,KY,40501,South,161045
3,F,41.70,Q1,H1,2003,14.43,16%,Munster,Lake,Munster,IN,46321,Midwest,142616
4,F,22.39,Q4,H2,2016,0.79,11%,Fairbanks,Fairbanks North Star,Fairbanks,AK,99709,West,135706


In [13]:
data.tail()

,Gender,Age in Yrs.,Quarter of Joining,Half of Joining,Year of Joining,Age in Company (Years),Last % Hike,Place Name,County,City,State,Zip,Region,Salary
995,F,32.01,Q1,H1,2008,9.56,10%,Mukwonago,Waukesha,Mukwonago,WI,53149,Midwest,72930
996,M,30.48,Q1,H1,2010,7.40,16%,Freeport,Barry,Freeport,MI,49325,Midwest,121949
997,F,59.88,Q1,H1,2001,16.44,14%,Turlock,Stanislaus,Turlock,CA,95381,West,82734
998,F,30.21,Q1,H1,2012,5.37,9%,Stamford,Fairfield,Stamford,CT,6910,Northeast,66854
999,F,59.78,Q1,H1,1980,37.55,6%,Meno,Major,Meno,OK,73760,South,105069
